In [87]:
import pandas as pd
import numpy as np
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score, multilabel_confusion_matrix


# 71 класс SCP_ECG в алфавитном порядке
labels = ['1AVB', '2AVB', '3AVB', 'ABQRS', 'AFIB', 'AFLT', 
          'ALMI', 'AMI', 'ANEUR', 'ASMI', 'BIGU', 'CLBBB', 
          'CRBBB', 'DIG', 'EL', 'HVOLT', 'ILBBB', 'ILMI', 
          'IMI', 'INJAL', 'INJAS', 'INJIL', 'INJIN','INJLA', 
          'INVT', 'IPLMI', 'IPMI', 'IRBBB', 'ISC_', 'ISCAL', 
          'ISCAN', 'ISCAS', 'ISCIL', 'ISCIN', 'ISCLA', 'IVCD', 
          'LAFB', 'LAO/LAE', 'LMI', 'LNGQT', 'LOWT', 'LPFB', 
          'LPR', 'LVH', 'LVOLT', 'NDT', 'NORM', 'NST_', 
          'NT_', 'PAC', 'PACE', 'PMI', 'PRC(S)', 'PSVT', 
          'PVC', 'QWAVE', 'RAO/RAE', 'RVH', 'SARRH', 'SBRAD', 
          'SEHYP', 'SR', 'STACH', 'STD_', 'STE_', 'SVARR', 
          'SVTAC', 'TAB_', 'TRIGU', 'VCLVH', 'WPW']

In [108]:
# Подсчет метрик f1, f2, g1 и g2 (где 1 и 2 значения параметра beta) для каждого класса в задаче многозначной классификации (multilabel classication)
def calc_metrics(y_true, y_pred, thresholds, labels): 
  num_classes = y_true.shape[1] # n x m, где m - количество классов (применялось one-hot кодирование)
  table_scores = pd.DataFrame({'label': labels[:num_classes], 
                               'f1': pd.Series([]), 
                               'f2': pd.Series([]), 
                               'g1': pd.Series([]), 
                               'g2': pd.Series([])}, index = range(1, num_classes + 1))
    
  for i, conf_matr in enumerate(multilabel_confusion_matrix(y_true, y_pred > thresholds)):
    
    label_index = i + 1 # i + 1, потому что в table_scores индексация начинается с 1                            
    true = y_true[:, i]
    pred = y_pred[:, i]
    table_scores.loc[label_index, 'f1'] = fbeta_score(true, pred > thresholds[i], beta = 1) 
    table_scores.loc[label_index, 'f2'] = fbeta_score(true, pred > thresholds[i], beta = 2)
                                
    TP = conf_matr[1, 1]
    FP = conf_matr[0, 1]
    TN = conf_matr[0, 0]
    FN = conf_matr[1, 0]                        
    beta = 1
    g1 = TP/(TP + FP + beta * FN) # beta = 1
    beta = 2
    g2 = TP/(TP + FP + beta * FN) # beta = 2
    table_scores.loc[label_index, 'g1'] = g1
    table_scores.loc[label_index, 'g2'] = g2
                                
  return table_scores

In [109]:
y_test = np.array([[1, 0, 1],
                   [0, 1, 0],
                   [1, 1, 0]])

y_pred = np.array([[0.8, 0.6, 0.3],
                   [0.4, 0.7, 0.9],
                   [0.2, 0.5, 0.6]])

lstm_thresholds = np.array([0.5, 0.6, 0.4])

In [110]:
table = calc_metrics(y_test, y_pred, lstm_thresholds, labels)

C:\Users\manuk\AppData\Local\Temp\ipykernel_11408\3998807189.py:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  'f1': pd.Series([]),
C:\Users\manuk\AppData\Local\Temp\ipykernel_11408\3998807189.py:6: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  'f2': pd.Series([]),
C:\Users\manuk\AppData\Local\Temp\ipykernel_11408\3998807189.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  'g1': pd.Series([]),
C:\Users\manuk\AppData\Local\Temp\ipykernel_11408\3998807189.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  'g2': pd.Series([

In [111]:
table

,label,f1,f2,g1,g2
1,1AVB,0.666667,0.555556,0.5,0.333333
2,2AVB,0.666667,0.555556,0.5,0.333333
3,3AVB,0.000000,0.000000,0.0,0.000000


In [66]:
table.loc[1] # вернуть строку по названию индекса (в данном случае индекс это номер класса, начиная с 1)

label    1AVB
f1        NaN
f2        NaN
g1        NaN
g2        NaN
Name: 1, dtype: object

In [67]:
table.loc[[1, 2]]

,label,f1,f2,g1,g2
1,1AVB,NaN,NaN,NaN,NaN
2,2AVB,NaN,NaN,NaN,NaN


In [68]:
table.iloc[0] # вернуть строку по номеру индекса (начиная с 0)

label    1AVB
f1        NaN
f2        NaN
g1        NaN
g2        NaN
Name: 1, dtype: object

In [76]:
table['f1']

1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
      ..
67   NaN
68   NaN
69   NaN
70   NaN
71   NaN
Name: f1, Length: 71, dtype: float64

In [77]:
table['f1'].loc[1]

nan

In [80]:
table.loc[1, 'f1'] = 1. # изменение ячейки в оригинальном dataframe.
table['f1'][1]

1.0